In [6]:
import pandas as pd
import os
from models import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
path = './osfstorage-archive/Experiment 1/twostep_data_study1/'
paths = [path + x for x in os.listdir(path) if x.endswith('.csv')]

In [3]:
def read_fn(fn):
    with open(fn) as f:
        lines = f.readlines()
    lines = [x.strip() for x in lines]
    i = min([i for i, x in enumerate(lines) if 'twostep_instruct_9' in x])
    lines = lines[i:]
    lines = [x.split(',') for x in lines]

    df = pd.DataFrame(lines[1:])
    return df

fn = [x for x in paths if '3A1COHJ8NJVPKFSRS1WC7SOUCMHH84' in x][0]
fn

df = read_fn(fn)


'./osfstorage-archive/Experiment 1/twostep_data_study1/3A1COHJ8NJVPKFSRS1WC7SOUCMHH84.csv'

In [4]:
# rename columns
"""A = trial_num
B = drift 1 (probability of reward after second stage option 1)
C = drift 2 (probability of reward after second stage option 2)
D = drift 3 (probability of reward after second stage option 3)
E = drift 4 (probability of reward after second stage option 4)
F = stage 1 response (left/right)
G = stage 1 selected stimulus (1/2 - note this is redundant with the response as the stage 1 options do not switch locations)
H = stage 1 RT
I = transition (common = TRUE; rare = FALSE)
J = stage 2 response (left/right)
K = stage 2 selected stimulus (1/2 - note this is redundant with response as the stage 2 options also do not switch locations)
L =  stage 2 state (identity 2 or 3)
M = stage 2 RT
N = reward (1= yes; 0=no)
O = redundant task variable, always set to 1"""

columns = """A = trial_num
B = drift1
C = drift2
D = drift3
E = drift4
F = choice1
G = choice1_int
H = stage1_RT
I = common_trans
J = choice2
K = choice2_int
L = stage2
M = stage2_RT
N = reward
O = redundant"""
columns = columns.split('\n')
columns = [x.split(' = ')[1] for x in columns]

# fn = [x for x in paths if '3A1COHJ8NJVPKFSRS1WC7SOUCMHH84' in x][0]
fn = paths[7]
df = read_fn(fn)
df.columns = columns
df.drop('redundant', axis=1, inplace=True)


df.loc[:, ['drift1', 'drift2', 'drift3', 'drift4']] = df.loc[:, ['drift1', 'drift2', 'drift3', 'drift4']].astype(float)

# drop rows where reward is -1. These seem to be where the participant did not respond in time, i.e. RT=2502?
df.reward = df.reward.astype(int)
df = df[df.reward != -1]

# reset index
df.reset_index(drop=True, inplace=True)

df.common_trans = df.common_trans.apply(lambda x: x == 'true')
df.choice1_int = df.choice1_int.astype(int) - 1
df.choice2_int = df.choice2_int.astype(int) - 1
df.stage2 = df.stage2.astype(int) - 2

'A = trial_num\nB = drift 1 (probability of reward after second stage option 1)\nC = drift 2 (probability of reward after second stage option 2)\nD = drift 3 (probability of reward after second stage option 3)\nE = drift 4 (probability of reward after second stage option 4)\nF = stage 1 response (left/right)\nG = stage 1 selected stimulus (1/2 - note this is redundant with the response as the stage 1 options do not switch locations)\nH = stage 1 RT\nI = transition (common = TRUE; rare = FALSE)\nJ = stage 2 response (left/right)\nK = stage 2 selected stimulus (1/2 - note this is redundant with response as the stage 2 options also do not switch locations)\nL =  stage 2 state (identity 2 or 3)\nM = stage 2 RT\nN = reward (1= yes; 0=no)\nO = redundant task variable, always set to 1'

In [7]:
drifts = df.loc[:, ['drift1', 'drift2', 'drift3', 'drift4']]
drifts_org = drifts.apply(lambda x: [[x[0], x[1]], [x[2], x[3]]], axis=1)
outs = []
outs2 = []
for i, x in df.iterrows():
    stage2 = x['stage2']
    choice2 = x['choice2_int']
    # reward prob of one we chose, and of the one we didn't
    outs.append([drifts_org.iloc[i][stage2][choice2], drifts_org.iloc[i][stage2][cbar(choice2)]])
    # and for other stage
    outs2.append([drifts_org.iloc[i][cbar(stage2)][choice2], drifts_org.iloc[i][cbar(stage2)][cbar(choice2)]])

outs = np.array(outs)
outs2 = np.array(outs2)
# if human is doing well, 0 should be > 1
outs.mean(axis=0)
# and generally better than other stage?
outs2.mean(axis=0)

# TODO why do these sometimes correlate and sometimes anticorrelate?

# this code 

array([0.54352428, 0.5060368 ])

array([0.50205791, 0.52139575])

In [96]:
drifts_org.iloc[0]

[[0.742621470936, 0.272532468277], [0.71669309271, 0.479056705494]]

In [6]:
from models import *

foo = Model()

In [7]:
[x for x in dir(foo) if not x.startswith('_')]

['Q_MB',
 'Q_MF0',
 'Q_MF1',
 'Qstage2',
 'alpha',
 'beta_stick',
 'get_stage1_action',
 'get_stage2_action']

In [9]:
foo.get_stage2_action(0)
foo.get_stage1_action(0)
foo.get_stage1_action(1)

array([0.5, 0.5])

array([0.73105858, 0.26894142])

array([0.26894142, 0.73105858])

In [9]:
foo.get_stage1_action(1)

array([1.        , 2.71828183])

In [9]:
foo.Q_MB.get_val(0)
foo.Q_MF0.get_val(0)
foo.Q_MF1.get_val(0)

0.0

0

0

In [10]:
foo.Q_MB.get_beta_scaled_val(0)
foo.Q_MF0.get_beta_scaled_val(0)
foo.Q_MF1.get_beta_scaled_val(0)

0.0

0.0

0.0

In [10]:
df_rw

,drift 1,drift 2,drift 3,drift 4
0,0.742621,0.272532,0.716693,0.479057
1,0.714054,0.289570,0.711770,0.456458
2,0.708948,0.346194,0.714403,0.465937
3,0.732490,0.339448,0.704013,0.385709
4,0.717249,0.407607,0.721518,0.422206
...,...,...,...,...
195,0.383793,0.712425,0.479336,0.387209
196,0.373070,0.734290,0.460442,0.375364
197,0.440371,0.747561,0.460608,0.410302
198,0.471404,0.736648,0.467873,0.375277


In [11]:
df

,trial_num,drift 1,drift 2,drift 3,drift 4,stage 1 response (left/right),stage 1 selected stimulus,stage 1 RT,transition (common TRUE; rare FALSE),stage 2 response (left/right),stage 2 selected stimulus,stage 2 state (identity 2 or 3),stage 2 RT,reward (1 yes; 0 no),"redundant task variable, always set to 1"
0,1,0.742621470936,0.272532468277,0.71669309271,0.479056705494,left,1,1096,false,right,2,3,484,1,1
1,2,0.714054437546,0.28956974681,0.711770360008,0.456458310278,left,1,384,false,right,2,3,291,0,1
2,3,0.708947909512,0.346194423994,0.714402963935,0.46593679168,left,1,284,true,left,1,2,269,1,1
3,4,0.732490347346,0.339447868367,0.704013189795,0.385709276536,right,2,343,true,left,1,3,418,1,1
4,5,0.717249489061,0.407606752597,0.721518196281,0.422206287292,right,2,311,false,left,1,2,316,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,0.383792753181,0.712424710636,0.479336007569,0.387209370348,left,1,1798,true,left,1,2,740,0,1
196,197,0.37306997392,0.734290462084,0.460441785745,0.375364189949,left,1,773,true,right,2,2,1148,1,1
197,198,0.440370694377,0.747560565418,0.460608262771,0.410302321559,left,1,337,false,right,2,3,389,1,1
198,199,0.471404410712,0.7366478523,0.467873357418,0.375276531813,left,1,345,true,right,2,2,366,1,1


In [13]:
df.iloc[:, 8]

0      false
1      false
2       true
3       true
4      false
       ...  
195     true
196     true
197    false
198     true
199     true
Name: transition (common TRUE; rare FALSE), Length: 200, dtype: object